# Data Loading

In [13]:
import pandas as pd 
from sklearn.model_selection import train_test_split

df = pd.read_csv("./input/melb_data.csv")
y = df['Price']
X = df.drop(['Price'],axis=1)

X_train_full,X_valid_full,y_train,y_valid = train_test_split(X,
                                                             y,
                                                             train_size=0.8,
                                                             test_size=0.2,
                                                             random_state=0)


*cadinality (n) : lực lượng (số lượng các phần tử riêng biệt)*

In [14]:
#Select CATEGORICAL cols with low cardinality
categorical_cols = [cname for cname in X_train_full.columns 
                    if X_train_full[cname].nunique() < 10 
                    and X_train_full[cname].dtype == 'object']

#Select NUMERCICAL cols
numerical_cols = [cname for cname in X_train_full.columns 
                  if X_train_full[cname].dtype in ['int64','float64']]

#Combine them
expected_cols = categorical_cols + numerical_cols
X_train = X_train_full[expected_cols].copy()
X_valid = X_valid_full[expected_cols].copy()

In [15]:
X_train.head()

,Type,Method,Regionname,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
12167,u,S,Southern Metropolitan,1,5.0,3182.0,1.0,1.0,1.0,0.0,NaN,1940.0,-37.85984,144.9867,13240.0
6524,h,SA,Western Metropolitan,2,8.0,3016.0,2.0,2.0,1.0,193.0,NaN,NaN,-37.85800,144.9005,6380.0
8413,h,S,Western Metropolitan,3,12.6,3020.0,3.0,1.0,1.0,555.0,NaN,NaN,-37.79880,144.8220,3755.0
2919,u,SP,Northern Metropolitan,3,13.0,3046.0,3.0,1.0,1.0,265.0,NaN,1995.0,-37.70830,144.9158,8870.0
6043,h,S,Western Metropolitan,3,13.3,3020.0,3.0,1.0,2.0,673.0,673.0,1970.0,-37.76230,144.8272,4217.0


# Pipepilines
*Pipelines (n) : đường ống, quy trình*

**Pipelines** are a simple way to keep your data preprocessing and modeling code organized. 

Benefits of pipeplines :
1. **Cleaner Code**: Accounting for data at each step of preprocessing can get messy. With a pipeline, you won't need to manually keep track of your training and validation data each step.
2. **Fewer Bugs**: 
3. **Easier to Productionize**:
4. **More Options for Model Validation**:

# Construct the pipeline
We construct the full pipeline in **three steps**.
1. Define Preprocessing Steps
2. Define the Model 
3. Create and Evaluate the Pipeline

## Step1 : Define Preprocessing Steps
*bungdle (v) : bó lại, kết hợp lại*

Using `ColumnTransformer` class to bundle together different preprocessing steps. The code below:
+ imputes missing values in **numerical** data, and
+ imputes missing values and applies a one-hot decoding to **categorical** data

In [23]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

## Step 2 : Define the Model

I choose `RandomForestRegressor`

In [21]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100,random_state=0)

## Step 3 : Create and Evaluate the Pipeline

Using the `Pipeline` class to define a pipeline that bundles preprocessing and modeling steps.
+ With the pipeline, we preprocess the training data and fit the model in a single line of code.
+ With the pipeline, we supply the unprocessed features in `X_valid` to the `predict()` command, and the pipeline automatically preprocesses the features before generating predictions. 

In [24]:
from sklearn.metrics import mean_absolute_error as mae 

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[
    ('preprocessor',preprocessor),
    ('model',model)
])

# Preprocessing training data and fit model
my_pipeline.fit(X_train,y_train)

# Preprocessing validation data and predict
preds = my_pipeline.predict(X_valid)

score = mae(y_valid,preds)
print('MAE:',score)

MAE: 160679.18917034855


# Conclusion
Pipelines are valuable for cleaning up machine learning code and avoiding errors, and are especially useful for workflows with sophisticated data preprocessing.